In [1]:
import pandas as pd
import os
import numpy as np
from scipy.spatial.distance import pdist, squareform
from sklearn.impute import KNNImputer
pd.options.mode.chained_assignment = None
file_dir = r'C:\Users\zhxum\OneDrive - University of Rochester\UR\Fall 2022\DSCC 383\Capstone_MacroX\Processed_Gases_Official_Data'

In [ ]:
coor = pd.read_csv(r'C:\Users\zhxum\OneDrive - University of Rochester\UR\Fall 2022\DSCC 383\Capstone_MacroX\station_coordinates.csv',index_col=0)
coor['file_name'] = os.listdir(file_dir)
coor['city'] = [name.split('_')[1] for name in os.listdir(file_dir)]
coor['coords'] = [[coor.Longitude[i], coor.Latitude[i]] for i in range(len(coor))]
coordinates_array = np.array(list(coor.coords))
dist_array = pdist(coordinates_array)
dist_matrix = squareform(dist_array)
dist_matrix = pd.DataFrame(dist_matrix, columns=coor.file_name, index=coor.file_name)
dist_matrix.to_csv('dist_matrix.csv',index=True)

In [2]:
dist_matrix = pd.read_csv(r'C:\Users\zhxum\OneDrive - University of Rochester\UR\Fall 2022\DSCC 383\Capstone_MacroX\Missing Value\dist_matrix.csv',index_col=0)

In [3]:
na_rate_list = []
for name in os.listdir(file_dir):
    df = pd.read_csv(file_dir+'\\'+name,index_col=0)
    df = df[:1687]
    na_rate = df.isna().sum().values.sum()/(4*len(df))
    na_rate_list.append(na_rate)
na_sort = pd.DataFrame({'file_name': os.listdir(file_dir), 'na_rate': na_rate_list})
na_sort.sort_values(by='na_rate',inplace=True)
na_sort.reset_index(drop=True, inplace=True)

In [3]:
na_sort

,file_name,na_rate
0,"Delhi_Delhi_NSIT Dwarka, Delhi - CPCB.csv",0.002075
1,"Delhi_Delhi_IHBAS, Dilshad Garden, Delhi - CPC...",0.002964
2,"Delhi_Delhi_Shadipur, Delhi - CPCB.csv",0.003112
3,"Delhi_Delhi_Mandir Marg, Delhi - DPCC.csv",0.009336
4,"Delhi_Delhi_Punjabi Bagh, Delhi - DPCC.csv",0.012596
...,...,...
178,"Meghalaya _Shillong_JN Stadium, Shillong - Meg...",0.967250
179,"Telangana_Hyderabad_ECIL Kapra, Hyderabad - TS...",0.972881
180,"Karnataka_Belgaum_Ramteerth Nagar, Belgaum - K...",0.995406
181,Tamil Nadu_Chennai_Gandhi Nagar.csv,0.996888


In [6]:
knn_imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
filled = na_sort['file_name'][na_sort.na_rate<0.02]
filled = set(filled)
for f in na_sort['file_name'][na_sort.na_rate<0.02]:
    df = pd.read_csv(file_dir+'\\'+f,index_col=0)
    df = df[:1687]
    df_knn = df.filter(['PM2.5','SO2','NO2','CO'], axis=1).copy()
    df_knn_imputed = pd.DataFrame(knn_imputer.fit_transform(df_knn), columns=df_knn.columns)
    df[['PM2.5','SO2','NO2','CO']] = df_knn_imputed
    file_path = os.path.join(r'C:\Users\zhxum\OneDrive - University of Rochester\UR\Fall 2022\DSCC 383\Capstone_MacroX\Missing Value\Distance Fill', f)
    df.to_csv(file_path)

In [9]:
def dist_fill(file_dir, na_sort, dist_matrix, station, filled):
    df = pd.read_csv(file_dir+'\\'+station,index_col=0)
    df = df[:1687]
    df_dist = dist_matrix[station].reset_index()
    df_dist.sort_values(by=station,inplace=True)
    df_dist.reset_index(drop=True, inplace=True)
    na_num = df.isna().sum().values.sum()
    for j in range(1, len(df_dist)):
        if na_num == 0:
            break
        if j > 9:
            if df_dist.file_name[j] in filled:
                fd = r'C:\Users\zhxum\OneDrive - University of Rochester\UR\Fall 2022\DSCC 383\Capstone_MacroX\Missing Value\Distance Fill'
                df_close = pd.read_csv(fd+'\\'+df_dist.file_name[j],index_col=0)
                df_close = df_close[:1687]
                for k in range(len(df_close)):
                    if np.isnan(df['CO'][k]):
                        df['CO'][k] = df_close['CO'][k]
                    if np.isnan(df['PM2.5'][k]):
                        df['PM2.5'][k] = df_close['PM2.5'][k]
                    if np.isnan(df['SO2'][k]):
                        df['SO2'][k] = df_close['SO2'][k]
                    if np.isnan(df['NO2'][k]):
                        df['NO2'][k] = df_close['NO2'][k]
                filled.add(df_dist.file_name[j])
                break
            continue
        else:
            df_close = pd.read_csv(file_dir+'\\'+df_dist.file_name[j],index_col=0)
            df_close = df_close[:1687]
            for k in range(len(df_close)):
                if np.isnan(df['CO'][k]):
                    if not np.isnan(df_close['CO'][k]):
                        df['CO'][k] = df_close['CO'][k]
                if np.isnan(df['PM2.5'][k]):
                    if not np.isnan(df_close['PM2.5'][k]):
                        df['PM2.5'][k] = df_close['PM2.5'][k]
                if np.isnan(df['SO2'][k]):
                    if not np.isnan(df_close['SO2'][k]):
                        df['SO2'][k] = df_close['SO2'][k]
                if np.isnan(df['NO2'][k]):
                    if not np.isnan(df_close['NO2'][k]):
                        df['NO2'][k] = df_close['NO2'][k]
            na_num = df.isna().sum().values.sum()
    file_path = os.path.join(r'C:\Users\zhxum\OneDrive - University of Rochester\UR\Fall 2022\DSCC 383\Capstone_MacroX\Missing Value\Distance Fill', station)
    return df.to_csv(file_path)

In [10]:
for f in na_sort['file_name'][na_sort.na_rate>=0.02]:
    dist_fill(file_dir, na_sort, dist_matrix, f, filled)

In [11]:
def dist_fill(file_dir, na_sort, dist_matrix, station, filled):
    df = pd.read_csv(file_dir+'\\'+station,index_col=0)
    df = df[:1687]
    df_dist = dist_matrix[station].reset_index()
    df_dist.sort_values(by=station,inplace=True)
    df_dist.reset_index(drop=True, inplace=True)
    na_num = df.isna().sum().values.sum()
    for j in range(1, len(df_dist)):
        if na_num == 0:
            break
        if j > 5:
            if df_dist.file_name[j] in filled:
                fd = r'C:\Users\zhxum\OneDrive - University of Rochester\UR\Fall 2022\DSCC 383\Capstone_MacroX\Missing Value\Cluster Fill'
                df_close = pd.read_csv(fd+'\\'+df_dist.file_name[j],index_col=0)
                df_close = df_close[:1687]
                for k in range(len(df_close)):
                    if np.isnan(df['CO'][k]):
                        df['CO'][k] = df_close['CO'][k]
                    if np.isnan(df['PM2.5'][k]):
                        df['PM2.5'][k] = df_close['PM2.5'][k]
                    if np.isnan(df['SO2'][k]):
                        df['SO2'][k] = df_close['SO2'][k]
                    if np.isnan(df['NO2'][k]):
                        df['NO2'][k] = df_close['NO2'][k]
                filled.add(df_dist.file_name[j])
                break
            continue
        else:
            df_close = pd.read_csv(file_dir+'\\'+df_dist.file_name[j],index_col=0)
            df_close = df_close[:1687]
            for k in range(len(df_close)):
                if np.isnan(df['CO'][k]):
                    if not np.isnan(df_close['CO'][k]):
                        df['CO'][k] = df_close['CO'][k]
                if np.isnan(df['PM2.5'][k]):
                    if not np.isnan(df_close['PM2.5'][k]):
                        df['PM2.5'][k] = df_close['PM2.5'][k]
                if np.isnan(df['SO2'][k]):
                    if not np.isnan(df_close['SO2'][k]):
                        df['SO2'][k] = df_close['SO2'][k]
                if np.isnan(df['NO2'][k]):
                    if not np.isnan(df_close['NO2'][k]):
                        df['NO2'][k] = df_close['NO2'][k]
            na_num = df.isna().sum().values.sum()
    file_path = os.path.join(r'C:\Users\zhxum\OneDrive - University of Rochester\UR\Fall 2022\DSCC 383\Capstone_MacroX\Missing Value\Cluster Fill', station)
    return df.to_csv(file_path)

In [ ]:
for f in filled:
    old_file = os.path.join(r'C:\Users\zhxum\OneDrive - University of Rochester\UR\Fall 2022\DSCC 383\Capstone_MacroX\Missing Value\fill_res', f)
    new_file = os.path.join(r'C:\Users\zhxum\OneDrive - University of Rochester\UR\Fall 2022\DSCC 383\Capstone_MacroX\Missing Value\Cluster Fill', '_'.join(f.split("_")[:-1])+'.csv')
    os.rename(old_file, new_file)

In [12]:
filled = os.listdir(r'C:\Users\zhxum\OneDrive - University of Rochester\UR\Fall 2022\DSCC 383\Capstone_MacroX\Missing Value\fill_res')
na_sort = na_sort[~na_sort['file_name'].isin(filled)]
filled = set(filled)

In [13]:
for f in na_sort['file_name']:
    dist_fill(file_dir, na_sort, dist_matrix, f, filled)

In [6]:
na_sort[na_sort.na_rate<0.35]

,file_name,na_rate
0,"Delhi_Delhi_NSIT Dwarka, Delhi - CPCB.csv",0.002075
1,"Delhi_Delhi_IHBAS, Dilshad Garden, Delhi - CPC...",0.002964
2,"Delhi_Delhi_Shadipur, Delhi - CPCB.csv",0.003112
3,"Delhi_Delhi_Mandir Marg, Delhi - DPCC.csv",0.009336
4,"Delhi_Delhi_Punjabi Bagh, Delhi - DPCC.csv",0.012596
...,...,...
68,Maharashtra_Mumbai_Chhatrapati Shivaji Intl. A...,0.326912
69,"Maharashtra_Mumbai_Powai, Mumbai - MPCB.csv",0.330913
70,"Delhi_Delhi_IGI Airport (T3), Delhi - IMD.csv",0.333432
71,"Maharashtra_Mumbai_Vile Parle West, Mumbai - M...",0.337878
